## Alberi di regressione
Algoritmi di regressione molto professionalizzanti. Il sito web Kagool, che propone sfide in ML, viene dimostrato che richieda l'uso nel 90% dei casi di alberi di regressione.

### Caso pratico
Abbiamo un dataset contenente persone di cui conosciamo
- età
- genere
- occupazione
e altre, che sono dette *features* o variabili $X$ di input.
La variabile *target* $y$ da predire è quanto una persona apprezza i videogame.
Vogliamo creare una struttura ad albero per modellare il fenomeno, dove ogni nodo intermedio è un predicato relativo a una variabile in input
<img src="imgs/alberi.png" alt="alberi" width=500>
Questi **predicati** dividono le istanze in due sezioni: quelli che danno come risposta "sì" al predicato e quelli che danno come risposta "no". Ovviamente ogni nodo avrà dei nodi sottofigli con i rispettivi sottoalberi.
Ogni nodo foglia conterrà la risposta al quesito di partenza. Cioè questi nodi foglia conterranno tutte le istanze classificate sulla base degli $n$ quesiti o **predicati** e su ogni foglia andrà calcolata la regressione per stimare l'opportuno $y$ corrispondente. Essendo di regressione, l'obbiettivo è quello di minimizzare l'errore in questa predizione, questa volta in *ogni* foglia. Cioè in ogni foglia ci sarà la regressione fatta su tutte le istanze che sono finite in quella foglia, tentando di raggrupparle in modo che il risultato di tale regressione sia il più simile fra istanze della stessa foglia possibile. Ad esempio nell'immagine sopra farò regressione selettivamente su 
- Persone di età maggiore di 15 anni, che produrrà intuitivamente un valore -1 poiché poco interessati.
- Persone di età inferiore a 15 anni e femmine, score pari a +0.1.
- Persone di età inferiore a 15 anni e maschi, otterrà score più alto pari a +2 (quelli che 
gradiscono maggiormente i videogame).
Ancora non sappiamo nulla su come sono scelti i predicati e in che ordine o su quando si è arrivati alle foglie o bisogna ulteriormente proseguire.

### Come funziona 
Proviamo ora a comprendere il meccanismo di funzionamento di questi algoritmi, partendo da una versione più semplice (a una sola feature) per poi estenderla a due feature e infine al caso generale
#### Singola feature
Consideriamo ora un dataset composto da una sola variabile $X$ che crei un albero binario (cioè che quando si splitta ha sia nodo figlio destro che sinistro). Osservando la seguente immagine
<img src="imgs/alberireg.png" alt="albero binario" width=400>
si vede lo spazio di dati su un grafico sul quale l'asse delle $x$ è per i dati che abbiamo, cioè $X$ e $y$ per la variabile da predire. Quello che si fa è iniziare con una regressione a valore costante, dunque graficamente una retta ortogonale rispetto all'asse y (in grigio) per cui i dati vengono divisi con un certo errore (calcolabile con i minimi quadrati: cioè quanto le nostre $y$ predette si allontanano da quelle fornite come dati). Questa regressione iniziale costituisce la radice dell'albero, a questo punto l'algoritmo prosegue scegliendo un'istanza $s_1 \in X$ come elemento, potremmo dire di partizionamento del dataset. Cioè su di esso (la retta rossa) vengono suddivisi i dati e si esegue una regressione di un certo tipo a seconda di dove si trova un determinato dato rispetto a tale retta. Cioè 
- per ogni $x < s_1$ si esegue una regressione con l'elemento $y_1$.
- per ogni $x \geq s_1$ la regressione è fatta su $y_2$.
In poche parole è come se i due insiemi di dati fossero differenti, e dunque venissero due modelli completamente differenti. È evidente che in questo modo *l'aderenza del modello ai dati sarà per forza maggiore* rispetto a una regressione unica per tutto il dataset, poiché ciascuna regressione è fatta su un insieme specifico di dati.

#### Come scegliere la $x$ opportuna
In questo modello abbiamo un dataset composto da una sola feature $x$ quindi non dobbiamo preoccuparci, per il momento, di come scegliere la feature (o l'asse) su cui splittare i dati, tuttavia bisogna capire come scegliere il valore da dare a quella feature per splittare, in altre parole bisogna capire come scegliere $s_1$. Questo valore viene scelto in maniera tale da rendere minimo l'errore commesso dalla retta di regressione sui dati di valore inferiore a $s_1$ e quella commessa dalla retta di regressione per i valori maggiori uguali. In questo si cela anche il criterio d'arresto, infatti se l'errore commesso dalla retta in una delle due regioni create a partire dal primo split è troppo grande rispetto a quanto stabilito, si genera un'ulteriore split su un altro valore. Cioè si continua a dividere la regione ricorsivamente con lo stesso criterio appena visto, finché l'errore di regressione commesso su ogni regione (che concettualmente rappresenta una foglia dell'albero) è minore di un certo valore.
Ad esempio proseguendo con il dataset di prima, lo spazio di sinistra (cioè dei dati $x < s_1$) viene ulteriormente suddiviso secondo il predicato "$x < s_2$" il quale darà origine a due spazi (figli del nodo $x < s_1$)
1. dei valori inferiori a $s_2$ ($x < s_2$).
2. valori compresi fra $s_2$ e $s_1$ ($s_2 \leq x < s_1$).

in modo che le due regressioni cosi prodotte abbiano errore ancora inferiore. La cosa funziona poiché le due rette di regressione passano più vicine ai sottoinsiemi di dati della propria regione.
<img src="imgs/splitting.png" alt="Ulteriore suddivisione della regione di sinistra" width=400>
Analogamente nello spazio di destra rispetto a $s_1$, cioè tutti i valori $\geq s_1$, viene partizionato in due sottoregioni (due sotto figli) secondo il predicato "$x < s_3$" tali che
1. la prima conterrà i valori compresi fra $s_1$ e $s_3$
2. la seconda conterrà i valori maggiori di $s_3$.
<img src="imgs/splitting3.png" alt="Ulteriore suddivisione della regione di destra" width=400>

Al termine di questa procedura abbiamo ottenuto quatto foglie (o sottospazi del dataset), ciascuno con una propria retta di regressione che approssima entro un errore prestabilito, tale regione. 

#### Due feature
Se avessimo un dominio a due feature $x_1, x_2$ ci troveremmo in uno spazio a tre dimensioni, due per le feature e una per la variabile $y$ target.
<img src="imgs/alberiduefeature.png" alt="Rappresentazione grafica modello a due feature" width=150>
Escludendo la variabile target, il grafico dello spazio sarebbe il seguente 
<img src="imgs/alberiduefeaturebidim.png" alt="Rappresentazione grafica modello a due feature" width=200>
nel quale una prima partizione viene fatta su una delle due variabili, ad esempio $x_1 =$ età, splittando la regione in quei valori tali per cui $x_1 < t_1$ (dove per continuare l'esempio la soglia $t_1 = 15$ anni), che genera una retta verticale che divide lo spazio in due regioni e il secondo split viene fatto sulla seconda variabile $x_2 < t_2$ (con soglia $t_2 =$ maschio). Questo secondo split divide la regione sinistra del primo split, nuovamente in due regioni $R1, R2$. A questo punto la regione di destra rispetto alla prima soglia $t_1$ viene nuovamente suddivisa rispetto alla variabile $x_1$ su una soglia $t_3$ per cui avremo una regione in cui $t_1 \leq x < t_3$ e una in  cui $x \geq t_3$. La regione a destra di questo nuovo split viene ulteriormente divisa, questa volta rispetto alla variabile $x_2$ in una soglia $t_4$. 
Il risultato, dal punto di vista dell'albero binario sarà il seguente
<img src="imgs/alberiduefeaturebinario.png" alt="Rappresentazione binaria modello a due feature" width=250>
Poiché ci troviamo in uno spazio tridimensionale, in realtà il significato geometrico della regressione non è più una retta bensì un piano ortogonale a $y$.
Ad esempio nella seguente immagine
<img src="imgs/treeconsumi.png" alt="Rappresentazione grafica modello consumi" width=550>
non facciamo altro che stimare il consumo in miglia per gallone MPG ($y$) sulla base di due feature (sugli assi orizzontali) che sono potenza in cavalli e peso del veicolo ($X$). Vediamo che ogni istanza è un punto e l'albero di regressione è dato dall'insieme di tutti i piani di regressione (ortogonali rispetto alla variabile MPG da predire). In questo modo, affettando lo spazio con tante regressioni, si ottiene l'errore più piccolo possibile nel prevedere il consumo per ciascuno dei punti forniti in input.

#### Algoritmo greedy di base
L'algoritmo è definito come segue
- Sia $k = 0$ una variabile di iterazione (serve per tenere il conto degli split che facciamo e $S = \mathbb{R}^d$ lo spazio iniziale dei dati, dove $d =$ numero di variabili in input. Ovviamente, dunque, $d$ è il numero di assi dello spazio vettoriale in questione.
- L'algoritmo sarà il seguente 

def ***RegressionTree***$(k, S)$:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$k += 1$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;per ogni variabile di input $j = 1, \dots, d$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;divide lo spazio $S$ scegliendo per $j$ il valore di separazione $v_j \in \mathbb{R}$ t.c. 
$$
    S_{j, <} = \{i | x_{i,j} < v_j \}
$$

$$
    S_{j, \geq} = \{i | x_{i,j} \geq v_j \} 
$$

$$
\overline{y}_{<} = \frac{\sum_{i \in S_{j, <}}(y_i)}{|S_{j, <}|}
$$

$$
\overline{y}_{\geq} = \frac{\sum_{i \in S_{j, \geq}}(y_i)}{|S_{j, \geq}|}
$$

$$
MSE_j = \text{min }\sum_{i \in S_{j, <}} (y_i - \overline{y}_<)^2 + \sum_{i \in S_{j, \geq}} (y_i - \overline{y}_{\geq})^2
$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;sia $p$ la variabile in $1, \dots, d$ che ha ottenuto il minimo $MSE_p$ 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;se la condizione di stop è insoddisfatta

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ***return RegressionTree***$(k, S_{p, <}) \cup$ ***RegressionTree***$(k, S_{p, \geq})$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;altrimenti

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;***return***[($p, v_p$)]

In pratica itero su ogni variabile di input $j$ fra tutte le $d$ possibili. Per ogni $j$ divido lo spazio scegliendo il valore di separazione $v_